Neural Model Identification: \
Trains a neural network based on data from the physics model to use as the predictive model
Note: You need to install the matlab engine to be able to run matlab code from pythone

In [27]:
import matplotlib.pyplot as plt

In [28]:
import os
import pickle
import math
#from utils import kits
import time
import random 
import numpy as np
import tensorflow as tf
import scipy.io
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
#from skopt.utils import use_named_args
#from skopt import gp_minimize
#from skopt.space import Real, Categorical, Integer
from sklearn import preprocessing

In [29]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [30]:
import scipy.io
model_data = scipy.io.loadmat('Data.mat')

In [31]:
Zin=model_data.get("Zin")
Zout=model_data.get("Zout")

## Define Functions

In [32]:
def swish(x, beta=1):
    return x * sigmoid(beta * x)

def get_model_layers(nodes, layers, activation, learning_rate, training=None):
    if activation == 'swish':
        activation = swish
    else:
        activation = activation

    inputs = tf.keras.layers.Input(shape=(in_dim,))
  #  xadd = (np.array(inputs[:, :out_dim])).reshape(None,1)
   # print(inputs[:, :out_dim].shape)
    #print(xadd.shape)
    x = tf.keras.layers.Dense(units=nodes, activation=activation)(inputs)
    for i in range(layers - 1):  # at least one layer which has the parameter input as input
        x = tf.keras.layers.Dense(units=nodes, activation=activation)(x)

    #x = tf.keras.layers.Dropout(rate=0.1)(x, training=training)
    outputs = tf.keras.layers.Dense(out_dim, activation="linear")(x)
   # outputs = tf.keras.layers.Add()([outputs, inputs[:, 2:]])

    #outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]])
    # outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]]) NN4UQ
    #    out = keras.layers.Dense(units=3, activation= "linear")(added)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    optimizer = optimizers.Adagrad(lr=learning_rate)# amsgrad=True)
    model.compile(optimizer=optimizer,
                  loss='mean_absolute_error')
    return model

In [33]:
get_custom_objects().update({'swish': Activation(swish)})

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(Zin, Zout, shuffle=True, train_size=0.8, random_state=42)
in_dim = X_train.shape[1]
out_dim = Y_train.shape[1]

In [35]:
learning_rate=0.05#gp_result.x[0]
num_layers = 3 #gp_result.x[1]
num_nodes = 5 #gp_result.x[2]
batch_size=10 #gp_result.x[3]
epochs=200
activation='swish'
DropModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                             activation=activation, learning_rate=learning_rate,
                             training=True)  # True-activates dropout during prediction
DropModel.summary()
DropModel.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data = (X_test, Y_test))

DetModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                            activation=activation, learning_rate=learning_rate)
#name = '{}_{}_{}_{}'.format(num_nodes, num_layers, activation, learning_rate)
#DropModel.save(root + 'results/nc/' + name)
DetModel.set_weights(DropModel.get_weights())
score = DetModel.evaluate(X_test, Y_test, verbose=0)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 18        
Total params: 108
Trainable params: 108
Non-trainable params: 0
_________________________________________________________________
Train on 1280 samples, validate on 320 samples
Epoch 1/200
1280/1280 [==============================] - 0s 225us/sample - loss:

Epoch 68/200
1280/1280 [==============================] - 0s 76us/sample - loss: 0.0020 - val_loss: 0.0023
Epoch 69/200
1280/1280 [==============================] - 0s 70us/sample - loss: 0.0019 - val_loss: 0.0018
Epoch 70/200
1280/1280 [==============================] - 0s 74us/sample - loss: 0.0019 - val_loss: 0.0024
Epoch 71/200
1280/1280 [==============================] - 0s 75us/sample - loss: 0.0018 - val_loss: 0.0017
Epoch 72/200
1280/1280 [==============================] - 0s 78us/sample - loss: 0.0019 - val_loss: 0.0015
Epoch 73/200
1280/1280 [==============================] - 0s 81us/sample - loss: 0.0019 - val_loss: 0.0015
Epoch 74/200
1280/1280 [==============================] - 0s 78us/sample - loss: 0.0019 - val_loss: 0.0026
Epoch 75/200
1280/1280 [==============================] - 0s 77us/sample - loss: 0.0019 - val_loss: 0.0014
Epoch 76/200
1280/1280 [==============================] - 0s 81us/sample - loss: 0.0018 - val_loss: 0.0021
Epoch 77/200
1280/1280 [=============

Epoch 144/200
1280/1280 [==============================] - 0s 69us/sample - loss: 0.0014 - val_loss: 0.0012
Epoch 145/200
1280/1280 [==============================] - 0s 68us/sample - loss: 0.0014 - val_loss: 0.0011
Epoch 146/200
1280/1280 [==============================] - 0s 78us/sample - loss: 0.0014 - val_loss: 0.0014
Epoch 147/200
1280/1280 [==============================] - 0s 68us/sample - loss: 0.0014 - val_loss: 0.0010
Epoch 148/200
1280/1280 [==============================] - 0s 75us/sample - loss: 0.0014 - val_loss: 0.0015
Epoch 149/200
1280/1280 [==============================] - 0s 71us/sample - loss: 0.0014 - val_loss: 0.0011
Epoch 150/200
1280/1280 [==============================] - 0s 69us/sample - loss: 0.0014 - val_loss: 0.0011
Epoch 151/200
1280/1280 [==============================] - 0s 75us/sample - loss: 0.0013 - val_loss: 0.0011
Epoch 152/200
1280/1280 [==============================] - 0s 68us/sample - loss: 0.0014 - val_loss: 0.0014
Epoch 153/200
1280/1280 [===

In [39]:
weights = DropModel.get_weights() # returs a numpy list of weights

In [40]:
Wmat = np.array(weights)

/Users/gmakr/opt/anaconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [41]:
Wmat

array([array([[ 0.0540182 , -0.22426239, -0.21128078,  0.17198083,  0.69992465],
       [-0.2890429 , -0.15379326, -0.30570132,  0.32041022,  0.11724483],
       [-0.64209586,  1.1316199 , -1.0313386 , -1.1422344 ,  0.49717587],
       [-0.49146357,  0.6660653 ,  0.30010462, -0.19761059, -0.59852964],
       [ 0.05770618,  0.03716956,  0.5813193 , -0.378852  ,  0.3586881 ]],
      dtype=float32),
       array([ 0.14158288, -0.06963691, -0.02404975,  0.04431865, -0.04889023],
      dtype=float32),
       array([[-0.63315594, -0.44350305,  0.11886229, -0.65862876, -0.5878999 ],
       [ 0.31246722, -0.36899263, -0.24641709,  0.64415586,  0.81268775],
       [ 0.18332578, -0.74409664,  0.28998616, -0.25908226,  0.3551644 ],
       [-0.7368128 , -0.64183253,  0.5338752 , -0.4943763 ,  0.20632228],
       [-0.13778208,  0.5321643 ,  0.17719038, -0.50505537,  0.4386702 ]],
      dtype=float32),
       array([-0.01964559,  0.07024211,  0.00408798,  0.03033691, -0.01767153],
      dtype=float3

In [13]:
import pickle as pkl
fileObject = open("weights_initialization.npy", 'wb')
pkl.dump(Wmat, fileObject)
fileObject.close()

#file = open("weights_init.npy", "wb")
#np.save(file, Wmat)
#file.close

In [14]:
def predictive_model_resnet(zk,Wmat): 
    #Assuming you can pass extra arguments to the function for casadi, otherwsise
    #Number of hidden layers and nodes are given once when we get the optimal resnet
    #The Bayesian Optimization Loop will be changing 
    n_hidden_layers = 3
    n_nodes = 5
    model_input = zk.reshape(1,5)
    Wmat_shape = Wmat.shape[0]
    
    #Initialize from input 
    sum0 = np.matmul(zk,Wmat[0]) #Matrix multiplication weights*inputs
    sumbias0 = sum0 + Wmat[1] #Add bias to construct y = sum(wi*xi) + b
    #neuron_out = swish_eval(sum1bias)
    neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation, out = fact(y)
    #They are vector quantitites so you might need to change stuff a bit with symbolic vars
    print(neuron_out.shape)

    #Forward prediction in hidden layers
    for k in range(n_hidden_layers-1):
        sum0 = np.matmul(neuron_out,Wmat[2*k+2])
        sumbias0 = sum0 + Wmat[2*k+3]
        neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation

    #Final output layer with linear activation
    sum0 = np.matmul(neuron_out,Wmat[Wmat_shape-2])
    sumbias0 = sum0 + Wmat[Wmat_shape-1]
    neuron_out = sumbias0  #linear activation

    return neuron_out #+ model_input[0,2:5]

def plantDynamics(matlabEngine, xAll, z):
    # Define the parameters whose scope is limited to this function
    p1 = 82.3
    p2 = 0.5
    p3 = 0.9
    
    # Convert np arrays to list and then to matlab.double
    xMatlab = matlab.double(xAll.tolist())
    zMatlab = [matlab.double([z[0,0]]), matlab.double([z[0,1]])]
    
    # Call the BCR_Discrete function and convert it to np.array since it is matlab type intially
    xNext = np.array(matlabEngine.BCR_Discrete(xMatlab, zMatlab[0], zMatlab[1], p1, p2, p3))
    
    return xNext

In [15]:
zk=Zin[0,:].reshape(1,5)

In [16]:
zkplusone  = predictive_model_resnet(zk,Wmat)

(1, 5)


In [17]:
zk[0,0]

0.041132586755882355

In [18]:
print('Current State:', zk,'Future State', zkplusone)

Current State: [[0.04113259 0.48376889 0.1        0.01       0.01      ]] Future State [[0.10214564 0.01493383 0.00859389]]


In [19]:
initial_cond_plant =scipy.io.loadmat('init_cond.mat')

In [20]:
xo = initial_cond_plant.get("yo")

In [21]:
xo.shape

(1, 87)

In [22]:
x0 = xo[0,-3:]

For BCR_Discrete you must convert the states array to a list like below and then put it into matlab.double; \
Same for matlab.double for the scalars \
The other, fixed parameters are ok like that

In [23]:
zkplusone_plant = eng.BCR_Discrete(matlab.double(xo.tolist()),matlab.double([zk[0,0]]),matlab.double([zk[0,1]]),82.3,1.0,0.9)
zkfuture_plant = np.array(zkplusone_plant)
print(zkfuture_plant[0,-3:]) #Getting the last states

[0.0997541  0.01959978 0.01200781]


In [24]:
zz = plantDynamics(eng, xo, zk)
print(zz[0,-3:])

[0.09995733 0.01499571 0.01103071]


In [25]:
zkfuture_plant  = predictive_model_resnet(np.array([zk[0,0],zk[0,1],x0[0],x0[1],x0[2]]),Wmat)

(5,)


In [26]:
print(zk)
print(zkfuture_plant)

[[0.04113259 0.48376889 0.1        0.01       0.01      ]]
[0.10214564 0.01493383 0.00859389]


One of the issues here with the zkfuture_plant

When you are at the initial condition, or acetate and ethanol are very low, usually you will predict negative concentrations for acetate and ethanol. In other words, you will predict x[1], and x[2] <0 which is unphysical. In this case you can make the prediction equal to zero (because otherwise you might not get feasible solutions)

#Objective Function
Area = 0.002436;   % reactor cross-sectional area, m^2 \
L = 1.06;   % reactor length, m  \
Qmedia[j] = Area \times L \times Dk[j]  (Dk is u[0], i.e., dilution rate) \
Try to maximize sum (Qmedia[j]*x1[j] ) (m3/h * mmol/m^3)  \
Constraints:
Selectivity is defined as Ce/Ca or x2/x1 and I want it to be x2/x1 < 0.45 \
u[0] in (0.01, 0.25) \
u[1] in (0.0, 0.9) \ 
x0,x1,x2 > 0 
